# Hadoop
(and Hadoop streaming)

<center>
<img src='https://media.licdn.com/mpr/mpr/shrinknp_800_800/AAEAAQAAAAAAAAcYAAAAJDgwYjA0ZmViLTJiNzgtNGJmMS1iNjE0LWQ3MzhiZmNjNzNhMg.png'>
</center>

**MapReduce** is a completely different paradigm 

* Solving a certain subset of parallelizable problems 
    - around the bottleneck of ingesting input data from disk
* Traditional parallelism brings the data to the computing machine
    - Map/reduce does the opposite, it brings the compute to the data
* Input data is not stored on a separate storage system
* Data exists in little pieces 
    - and is permanently stored on each computing node

MapReduce is the programming paradigm that allows massive scalability across thousands of servers.

Its open source server implementation is the *Hadoop* cluster.

Also always keep in mind that ***HDFS*** is fundamental to Hadoop 

* it provides the data chunking distribution across compute elements 
* necessary for map/reduce applications to be efficient

# Our (*short*) road to real Hadoop

1. See how Java original Hadoop works
2. Copy files from the system to HDFS within the notebooks
3. Launch Hadoop
4. Understand and launch Hadoop Streaming
5. Simulate Hadoop streaming with Bash Pipes
6. Launch Hadoop Streaming with Python code

# Word count
The '`Hello World`' for MapReduce

Among the simplest of full Hadoop jobs you can run

<img src='http://www.glennklockwood.com/data-intensive/hadoop/wordcount-schematic.png'
width='700'>
<small>Reading ***Moby Dick*** </small>

### How it works
* The **MAP step** will take the raw text and convert it to key/value pairs
    - Each key is a word
    - All keys (words) will have a value of 1


* The **REDUCE step** will combine all duplicate keys 
    - By adding up their values (sum)
    - Every key (word) has a value of 1 (Map)
    - Output is reduced to a list of unique keys
    - Each key’s value corresponding to key's (word's) count

<center>
<img src='http://disco.readthedocs.org/en/latest/_images/map_shuffle_reduce.png' width=800>
</center>

### Map function:
processes data and generates a set of  intermediate key/value pairs.


### Reduce function:
merges all intermediate values  associated with the same intermediate key.

## A WordCount example 
*(with Java)*

Consider doing a word count of the following file using  MapReduce:
```
Hello World Bye World
Hello Hadoop Goodbye Hadoop
```

The map function reads in words one at a time outputs (“word”, 1) for each parsed input word

```
(Hello, 1)
(World, 1)
(Bye, 1)
(World, 1)
(Hello, 1)
(Hadoop, 1)
(Goodbye, 1)
(Hadoop, 1)
```

The shuffle phase between map and reduce creates a  list of values associated with each key
```
(Bye, (1))
(Goodbye, (1))
(Hadoop, (1, 1))
(Hello, (1, 1))
(World, (1, 1))
```

The reduce function sums the numbers in the list for each  key and outputs (word, count) pairs
```
(Bye, 1)
(Goodbye, 1)
(Hadoop, 2)
(Hello, 2)
(World, 2)
```

## How can you do this with Java?
(the Hadoop framework native language)

``` Java
// Imports
package org.myorg;
import java.io.IOException;
import java.util.*;
import org.apache.hadoop.*

// Create JAVA class
public class WordCount {
```

``` Java
//Mapper function
  public static class Map extends MapReduceBase implements Mapper<LongWritable, Text, Text, IntWritable> {
    private final static IntWritable one = new IntWritable(1);
    private Text word = new Text();

    public void map(LongWritable key, Text value, OutputCollector<Text, IntWritable> output, Reporter reporter) throws IOException {
      String line = value.toString();
      StringTokenizer tokenizer = new StringTokenizer(line);
      while (tokenizer.hasMoreTokens()) {
        word.set(tokenizer.nextToken());
        output.collect(word, one);
      }
    }
  }
```

``` Java
//Reducer function
  public static class Reduce extends MapReduceBase implements Reducer<Text, IntWritable, Text, IntWritable> {
    public void reduce(Text key, Iterator<IntWritable> values, OutputCollector<Text, IntWritable> output, Reporter reporter) throws IOException {
      int sum = 0;
      while (values.hasNext()) {
        sum += values.next().get();
      }
      output.collect(key, new IntWritable(sum));
    }
  }
    
```

<small>
``` Java
//Main function
  public static void main(String[] args) throws Exception {
    JobConf conf = new JobConf(WordCount.class);
    conf.setJobName("wordcount");

    conf.setOutputKeyClass(Text.class);
    conf.setOutputValueClass(IntWritable.class);

    conf.setMapperClass(Map.class);
    conf.setCombinerClass(Reduce.class);
    conf.setReducerClass(Reduce.class);

    conf.setInputFormat(TextInputFormat.class);
    conf.setOutputFormat(TextOutputFormat.class);

    FileInputFormat.setInputPaths(conf, new Path(args[0]));
    FileOutputFormat.setOutputPath(conf, new Path(args[1]));

    JobClient.runJob(conf);
  }
```
</small>

We can test the Java code here. *Live*.

In [ ]:
%env HADOOP_EXAMPLES /usr/local/hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.6.0.jar
%env HADOOP_STREAMING /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.6.0.jar

In [ ]:
%env HADOOP_EXAMPLES

In [ ]:
# Hadoop available examples
! hadoop jar $HADOOP_EXAMPLES | grep word

In [ ]:
# Check wordcount
! hadoop jar $HADOOP_EXAMPLES wordcount

## Brief Demo

In [ ]:
# Our input

! cat ./data/txt/twolines.txt

In [ ]:
%%bash

########################
# Preprocess with HDFS

# Create input directory
hdfs dfs -mkdir myinput
# Save one file inside
file="./data/txt/twolines.txt"
hdfs dfs -put $file myinput/file01
# Remove output or Hadoop will give error if existing
hdfs dfs -rm -r -f myoutput

In [ ]:
# Test wordcount with real hadoop on our system
! hadoop jar $HADOOP_EXAMPLES wordcount myinput myoutput

In [ ]:
# Hadoop output
! hadoop fs -cat myoutput/part*

# Recap
<img src='https://pbs.twimg.com/media/B2RlCy-IIAEFCLC.jpg' width=700>

# Hadoop like `pipes` in Unix Bash

Prepare a data sample

**Warning**: please run this code on your notebook too :)

In [ ]:
# Variables for python and bash
myfile = '/tmp/ngs.sam'
%env myfile $myfile

In [ ]:
%%bash

# Download compressed NGS data from a link
wget -q "http://bit.ly/ngs_sample_data" -O $myfile.bz2 && echo "downloaded"
# Decompress the file
bunzip2 $myfile.bz2 && echo "decompressed"

In [ ]:
# Check if the file is there
! ls $myfile

In [ ]:
%%bash
# Bash piping our own MapReduce with Unix commands
head -2000 $myfile | tail -n 10 | awk '{ print $3":"$4 }' | sort | uniq -c

## Understand it better

Splitting the command:

1. `head -2000 data/ngs/input.sam | tail -n 10`

1. `awk '{ print $3":"$4 }’`

1. `sort`

1. `uniq -c`


**INPUT STREAM**
`head -2000 data/ngs/input.sam | tail -n 10`

**MAPPER**
`awk '{ print $3":"$4 }'`

**SHUFFLE**
`sort`

**REDUCER**
`uniq -c`

**OUTPUT STREAM**
`<STDOUT>`

Let's move step by step and see what happens

In [ ]:
# STREAMING THE FILE
! head -2000 $myfile | tail -n 10
# note: 
# taking the last 10 lines of the first 2000
# to skip headers lines

In [ ]:
# MAPPING
! head -2000 $myfile | tail -n 10 | awk '{ print $3":"$4 }'

In [ ]:
# SHUFFLING
! head -2000 $myfile | tail -n 10 | awk '{ print $3":"$4 }' | sort

In [ ]:
# REDUCER
! head -2000 $myfile | tail -n 10 | awk '{ print $3":"$4 }' | sort | uniq -c

# Exercise
<br>

<big>
Play *a little bit* with bash pipes. 
</big>

Try to understand how MapReduce process data

### Considerations with bash pipes as simulation of MapReduce

* Serial steps
* No file distribution
* Single node
* Single mapper
* Single reducer
* Can we add a Combiner?

# Exercise

Do you know how to substitute the awk/mapper with a python script?

If yes, create one to count the most recurring words inside the Divine Comedy.

# Hadoop streaming
### Concepts and mechanisms

Hadoop streaming is a utility 

* It comes bundled with the Hadoop distribution
* It allows creating and running Map/Reduce jobs 
    - with any executable or script as the mapper and/or the reducer

Protocol steps

* Create a Map/Reduce job
* Submit the job to an appropriate cluster
* Monitor the progress of the job until it completes
* Links to Hadoop HDFS job directories

### Why?

One of the most unappetizing aspects of Hadoop to users of traditional HPC is that it is written in Java. 

* Java is not originally designed to be a high-performance language
* Learning Java is very difficult for domain scientists

This is why Hadoop allows you to write map/reduce code in any language you want using the Hadoop Streaming interface

* It means turning an existing Python or Perl script into a Hadoop job
* Does not require learning any Java at all

### MapReduce streaming with binaries/executables

* Executables are specified for mappers and reducers!
    - each mapper task run as a separate process 
* Inputs converted into lines and feed to the `STDIN` of the process
* The mapper collects `STDOUT` of the process 
    - each line is a key/value pair **separated by TAB**
    - e.g. ”this is the key\tvalue is the rest\n”

warning: If there is no tab character in the line, then entire line is considered as key and the value is null (!)

## Let's do some live experiments...

A streaming command line example:
``` bash
$ hadoop jar $HADOOP_HOME/hadoop-streaming.jar \
    -input myInputDirs \
    -output myOutputDir \
    -mapper org.apache.hadoop.mapred.lib.IdentityMapper \
    -reducer /bin/wc
```

A streaming command line example **for python**:
``` bash
$ hadoop jar $HADOOP_HOME/hadoop-streaming.jar \
    -files mapper.py,reducer.py
    -input input_dir/ \
    -output output_dir/ \
    -mapper mapper.py \
    -reducer reducer.py \
```

Before submitting the Hadoop Streaming job:

* Make sure your scripts have no errors
* Do mapper and reducer scripts actually work?

This is just a matter of running them through pipes on a **little bit** of sample data,

like `cat` or `head` linux bash commands, with pipes, as seen before.

```
# Simulating hadoop streaming with bash pipes
$ cat $file | python mapper.py | sort | python reducer.py
```

First approach: split

In [ ]:
%%writefile mapper.py

# -*- coding: utf-8 -*-
import sys

for line in sys.stdin:
    line = line.strip()
    pieces = line.split('\t')
    print(pieces) 


In [ ]:
! head -n 150 $myfile | python mapper.py

# Exercise

Count symbols (everything that is not a letter of the alphabet) inside a text file.

Back to the example:

Skip header lines.

In [ ]:
%%writefile mapper.py

# -*- coding: utf-8 -*-
TAB = "\t"
import sys

# Cycle current streaming data
for line in sys.stdin:

    # Clean input
    line = line.strip()
    # Skip SAM/BAM headers
    if line[0] == "@":
        continue

    # Use data
    pieces = line.split(TAB)
    mychr = pieces[2]
    mystart = int(pieces[3])
    myseq = pieces[9]
    print(mychr,mystart.__str__())
    sys.exit(1)

In [ ]:
! head -n 100 $myfile | python mapper.py

Produce an output that can be sorted and then used by reducer

In [ ]:
%%writefile mapper.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
TAB = "\t"
SEP = ':'
import sys

# Cycle current streaming data
for line in sys.stdin:
    # Clean input
    line = line.strip()
    # Skip SAM/BAM headers
    if line[0] == "@":
        continue
    
    # Use data
    pieces = line.split(TAB)
    mychr = pieces[2]
    mystart = int(pieces[3])
    myseq = pieces[9]

    mystop = mystart + len(myseq)

    # Each element with coverage
    for i in range(mystart,mystop):
        results = [mychr+SEP+i.__str__(), "1"]
        print(TAB.join(results))


In [ ]:
! head -n 100 $myfile | python mapper.py | tail

### Shuffle step 

<br><big>
A lot happens, transparent to the developer
</big>

* Mappers’s output is transformed and distributed to the reducers
* All key/value pairs are sorted before sent to reducer function
* Pairs sharing the same key are sent to the same reducer
* If you encounter a key that is different from the last key you processed
    - *you know that previous key will never appear again*
* If your keys are all the same
    - only use one reducer and gain no parallelization
    - come up with a more unique key if this happens

### Reducer

In [ ]:
%%writefile reducer.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
TAB = "\t"
SEP = ':'
import sys
last_value = ""
value_count = 1
for line in sys.stdin:
    value, count = line.strip().split(TAB)
    # if this is the first iteration
    if not last_value:
        last_value = value
    # if they're the same, log it
    if value == last_value:
        value_count += int(count)
    else:
        # state change
        try: 
            print(TAB.join([last_value, str(value_count)]))
        except:
            pass
        last_value = value
        value_count = 1
# LAST ONE after all records have been received
print(TAB.join([last_value, str(value_count)]))

In [ ]:
%%bash
# needs ~ 5 seconds for running
time head -n 10000 $myfile | python mapper.py | sort | python reducer.py | head -n 5

In [ ]:
%%bash
time cat $myfile | python mapper.py | sort | python reducer.py > out.txt

In [ ]:
! grep "\s185" out.txt

# Moving to real Hadoop

<big>
A working python code tested on pipes **should work** with Hadoop Streaming
</big>

* To make this happen we need to handle copy of input and output files inside the Hadoop FS
* Also the job tracker logs will be found inside HDFS
* We are going to use bash scripting inside the notebook to make our workflow

## Preprocessing

HDFS commands to interact with Hadoop file system use the same syntax:

```
hdfs dfs -command
```

`command` are like bash commands for file

e.g.

```
hadoop fs -mkdir hdfs:///dir
hadoop fs -put file_on_host hdfs:///path/to/file
hadoop fs -ls
```

<small>
Note: we have seen this in action with the Java example
</small>

<big>
Hadoop Streaming needs “binaries” to execute
</big>

You need to specify interpreter at the beginning of your scripts:
```
#!/usr/bin/env python
```

Make also the script executables:
```
chmod +x hs*.py
```

In [ ]:
! chmod +x mapper.py reducer.py

In [ ]:
%env HADOOP_STREAMING

In [ ]:
%%bash
# Launch streaming
hadoop jar $HADOOP_STREAMING

In [ ]:
%%bash
# Preprocess with HDFS
hdfs dfs -rm -r -f myinput
hdfs dfs -mkdir myinput
# Save one file inside
file="/tmp/ngs.sam"
hdfs dfs -put $file myinput/file01
# Remove output or Hadoop will give error if existing
hdfs dfs -rm -r -f myoutput

Final launch via bash command for using Hadoop streaming

In [ ]:
%%bash 
# A real Hadoop Streaming run
time hadoop jar $HADOOP_STREAMING \
    -D mapreduce.job.mapper=12 -D mapreduce.job.reducers=4  \
    -files mapper.py,reducer.py \
    -input myinput -output myoutput \
    -mapper mapper.py -reducer reducer.py

Hadoop streaming is **difficult to debug**.
Just like real Java Hadoop.

If you did a typical setup mistake, you may end receiving unrelated errors stacktrace from the Java virtual machine.

So before googling those stacktrace, make sure that:

* Python files (mapper and reducer) exists
* They are provided inside the main bash command also as **files** list
* They are executables and contain as first line the hashbang
* Your input directory exists on HDFS
* Files inside your input directory are not corrupted
    - e.g. bad decompression

In [ ]:
# OUTPUT: Check directory
! hdfs dfs -ls myoutput

In [ ]:
# OUTPUT:  Copy file and go see it
! rm -rf hs.*.txt && hdfs dfs -get myoutput/part-00000 hs.out.txt

In [ ]:
! head hs.out.txt

## Final thoughts on Hadoop Streaming 


* Provides options to write MapReduce jobs in other languages
* Even executables can be used to work as a MapReduce job
* One of the best examples of flexibility available to MapReduce
* Fast
* Simpler than Java
* Also close to the original standard Java API Hadoop power


### Where it really works

* When the developer do not have knowhow of Java 
* Write Mapper/Reducer in any scripting language 

### Disadvantages

* Force scripts in a Java VM
    - Although almost free overhead
* The program/executable have to take input from STDIN 
    - and produce output at STDOUT
* Restrictions on the input/output formats
    - Does not take care of input and output file and directory preparation
    - User have to implement hdfs commands “hand-made”

### Where it falls short

* No pythonic way to work the MapReduce code

(Because it was not written specifically for python)

## Recap 

* Hadoop streaming handles Hadoop in almost a classic manner
* Wrap any executable (and script)
    - Also python scripts
* Runnable on a cluster using a non-interactive, all-encapsulated job

# End of this Chapter